In [5]:
pip install scikit-learn


^C
Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from pygam import LogisticGAM
from sklearn.metrics import mean_squared_error, r2_score


In [9]:
# Load data
data = pd.read_excel("data.xlsx")  
data.head(10)

,Platform,business_id,city,state,postal_code,score,review_count,Gender,CEO_sch_cat,CEO_grd_yr,...,ZIP Code,Business_ID_other,Rural_metropolitan_Desc,Tot_Clms_Services,Brnd_Tot_Clms_Services,Gnrc_Tot_Clms_Services,Othr_Tot_Clms_Services,LIS_Tot_Clms_Services,Opioid_Tot_Clms_Services,Antbtc_Tot_Clms_Services
0,Platform 1,1770720401,Santa Barbara,CA,93101,5.0,7,F,108,1997.0,...,931032109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Platform 1,1699268318,Clearwater,FL,33755,5.0,10,F,108,2017.0,...,337631726,1.699268e+09,Metropolitan area core: primary flow within an...,971.0,138.0,813.0,20.0,762.0,NaN,31.0
2,Platform 1,1336331669,Bala Cynwyd,PA,19004,4.0,13,M,108,1986.0,...,190043207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Platform 1,1528008463,Plymouth Meeting,PA,19462,2.5,8,M,138,1980.0,...,194621718,1.528008e+09,Metropolitan area core: primary flow within an...,1988.0,NaN,1674.0,NaN,65.0,NaN,274.0
4,Platform 1,1366867269,Voorhees,NJ,8043,3.5,17,F,116,2014.0,...,80434509,1.366867e+09,Metropolitan area core: primary flow within an...,847.0,73.0,774.0,0.0,75.0,50.0,NaN
5,Platform 1,1689735383,Tarpon Springs,FL,34689,2.0,29,M,170,1988.0,...,346893790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Platform 1,1356471593,Tampa,FL,33607,3.0,36,M,116,1992.0,...,805162422,1.356472e+09,Metropolitan area core: primary flow within an...,869.0,103.0,766.0,0.0,106.0,33.0,25.0
7,Platform 1,1023598968,Brownsburg,IN,46112,1.5,14,M,108,2018.0,...,461121031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Platform 1,1316037153,Tucson,AZ,85718,2.5,43,M,3,1992.0,...,104672401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Platform 1,1902048259,Saint Petersburg,FL,33713,1.5,6,M,201,2009.0,...,337051300,1.902048e+09,Metropolitan area core: primary flow within an...,24.0,11.0,13.0,0.0,0.0,0.0,0.0


### Explore data


In [16]:
data.columns

Index(['Platform', 'business_id', 'city', 'state', 'postal_code', 'score',
       'review_count', 'Gender', 'CEO_sch_cat', 'CEO_grd_yr', 'field_cat',
       'ZIP Code', 'Business_ID_other', 'Rural_metropolitan_Desc',
       'Tot_Clms_Services', 'Brnd_Tot_Clms_Services', 'Gnrc_Tot_Clms_Services',
       'Othr_Tot_Clms_Services', 'LIS_Tot_Clms_Services',
       'Opioid_Tot_Clms_Services', 'Antbtc_Tot_Clms_Services'],
      dtype='object')

In [13]:
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10891 entries, 0 to 10890
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Platform                  10891 non-null  object 
 1   business_id               10891 non-null  int64  
 2   city                      10891 non-null  object 
 3   state                     10891 non-null  object 
 4   postal_code               10890 non-null  object 
 5   score                     10891 non-null  float64
 6   review_count              10891 non-null  int64  
 7   Gender                    10891 non-null  object 
 8   CEO_sch_cat               10891 non-null  int64  
 9   CEO_grd_yr                10883 non-null  float64
 10  field_cat                 10891 non-null  int64  
 11  ZIP Code                  10891 non-null  int64  
 12  Business_ID_other         5972 non-null   float64
 13  Rural_metropolitan_Desc   5971 non-null   object 
 14  Tot_Cl

In [15]:
data.describe()


,business_id,score,review_count,CEO_sch_cat,CEO_grd_yr,field_cat,ZIP Code,Business_ID_other,Tot_Clms_Services,Brnd_Tot_Clms_Services,Gnrc_Tot_Clms_Services,Othr_Tot_Clms_Services,LIS_Tot_Clms_Services,Opioid_Tot_Clms_Services,Antbtc_Tot_Clms_Services
count,1.089100e+04,10891.000000,10891.000000,10891.000000,10883.000000,10891.000000,1.089100e+04,5.972000e+03,5972.000000,3381.000000,5905.000000,3370.000000,4993.000000,4437.000000,4104.000000
mean,1.508429e+09,3.491094,15.492058,119.815995,2003.567123,37.321458,4.556338e+08,1.498723e+09,1385.228567,313.112689,1162.889077,16.085757,637.495293,73.534145,72.947125
std,2.769638e+08,1.072750,19.330906,45.570808,12.408939,20.639548,2.779539e+08,2.734814e+08,2699.951669,580.514365,2295.809344,49.213090,1732.955977,242.297587,98.750373
min,1.003019e+09,1.000000,5.000000,0.000000,1956.000000,0.000000,6.488000e+03,1.003051e+09,11.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000
25%,1.285817e+09,2.500000,6.000000,108.000000,1997.000000,18.000000,1.934113e+08,1.265789e+09,98.000000,18.000000,85.000000,0.000000,43.000000,0.000000,20.000000
50%,1.508851e+09,3.500000,10.000000,108.000000,2004.000000,45.000000,3.702757e+08,1.497780e+09,337.000000,79.000000,268.000000,0.000000,120.000000,14.000000,38.000000
75%,1.760076e+09,4.500000,17.000000,134.000000,2014.000000,58.000000,7.011519e+08,1.720086e+09,1454.000000,364.000000,1181.000000,0.000000,572.000000,52.000000,96.000000
max,1.992998e+09,5.000000,413.000000,226.000000,2023.000000,76.000000,9.980178e+08,1.992953e+09,45742.000000,8888.000000,38693.000000,1218.000000,40132.000000,5317.000000,1499.000000


### Handle missing values

In [17]:
data.fillna(data.mean(), inplace=True)

C:\Users\Ibtasam Ahmad\AppData\Local\Temp\ipykernel_24504\1474580016.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.fillna(data.mean(), inplace=True)


### Separate target variable and features


In [18]:
target = data["score"]
features = data.drop("score", axis=1)

print(target, features)

0        5.0
1        5.0
2        4.0
3        2.5
4        3.5
        ... 
10886    4.5
10887    2.5
10888    3.5
10889    4.5
10890    4.5
Name: score, Length: 10891, dtype: float64          Platform  business_id              city state postal_code  \
0      Platform 1   1770720401     Santa Barbara    CA       93101   
1      Platform 1   1699268318        Clearwater    FL       33755   
2      Platform 1   1336331669       Bala Cynwyd    PA       19004   
3      Platform 1   1528008463  Plymouth Meeting    PA       19462   
4      Platform 1   1366867269          Voorhees    NJ        8043   
...           ...          ...               ...   ...         ...   
10886  Platform 1   1518938117     Santa Barbara    CA       93101   
10887  Platform 1   1649837246  Maryland Heights    MO       63043   
10888  Platform 1   1740616986              Reno    NV       89503   
10889  Platform 1   1437123254       Saint Louis    MO       63131   
10890  Platform 1   1306872635      Philadel

### Address categorical variables

In [20]:
categorical_features = [col for col in features.columns if features[col].dtype == object]
features = pd.get_dummies(features, columns=categorical_features)

print(categorical_features, features)

[]        business_id  review_count  CEO_sch_cat  CEO_grd_yr  field_cat  \
0       1770720401             7          108      1997.0         13   
1       1699268318            10          108      2017.0         21   
2       1336331669            13          108      1986.0         12   
3       1528008463             8          138      1980.0         75   
4       1366867269            17          116      2014.0         69   
...            ...           ...          ...         ...        ...   
10886   1518938117            15          128      1997.0         49   
10887   1649837246             7          108      2018.0         59   
10888   1740616986            20          108      2015.0         13   
10889   1437123254             7          126      2004.0          5   
10890   1306872635            39          108      1989.0         35   

        ZIP Code  Business_ID_other  Tot_Clms_Services  \
0      931032109       1.498723e+09        1385.228567   
1      337631726

### Split data for training and testing


In [21]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)


In [26]:
alphas = np.arange(0.01, 1, 0.01)  # Define range of regularization parameters
alphas

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [27]:
scores = []
for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    scores.append(cross_val_score(lasso, X_train, y_train, cv=5).mean())

scores

c:\Users\Ibtasam Ahmad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.728e+03, tolerance: 8.086e-01
  model = cd_fast.enet_coordinate_descent(
c:\Users\Ibtasam Ahmad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.649e+03, tolerance: 7.963e-01
  model = cd_fast.enet_coordinate_descent(
c:\Users\Ibtasam Ahmad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of i

### Select alpha with best cross-validation score


In [ ]:
best_alpha = alphas[np.argmax(scores)]
lasso_model = Lasso(alpha=best_alpha)
lasso_model.fit(X_train, y_train)

### Analyze Lasso model coefficients


In [ ]:
print(f"Lasso Model Coefficients:\n{lasso_model.coef_}")


### Feature importance visualization"

In [ ]:
plt.bar(features.columns, lasso_model.coef_)
plt.xlabel("Features")
plt.ylabel("Coefficient")
plt.title("Lasso Model Coefficients")
plt.xticks(rotation=45)
plt.show()

### Smooth Features

In [ ]:
smooth_features = ["Tot_Clms_ServicesBrnd", "Tot_Clms_ServicesGnrc"]  # Features for smooth terms
plt.scatter(data[smooth_features[0]], data["score"])
plt.title(f"Score vs. {smooth_features[0]}")
plt.show()

### Define GAM model with smooth terms for chosen features


In [ ]:
gam = LogisticGAM(smooth_terms=smooth_features)
gam

### Train the GAM model


In [ ]:
gam.fit(X_train, y_train)


### Prediction on test data


In [ ]:
gam_pred = gam.predict(X_test)


### Performance metrics

In [ ]:
lasso_mse = mean_squared_error(y_test, lasso_model.predict(X_test))
lasso_r2 = r2_score(y_test, lasso_model.predict(X_test))

gam_mse = mean_squared_error(y_test, gam_pred)
gam_r2 = r2_score(y_test, gam_pred)

In [ ]:
""# Print model performance comparison
print(f"Lasso MSE: {lasso_mse:.4f}, R-squared: {lasso_r2:.4f}")
print(f"GAM MSE: {gam_mse:.4f}, R-squared: {gam_r2:.4f}")

### Visualize smooth terms

In [ ]:
gam.plot_partial_dependence(smooth_features[0])
plt.title(f"Partial Dependence Plot: {smooth_features[0]}")
plt.show()

### Sort features by absolute coefficient values


In [ ]:
important_features = np.abs(lasso_model.coef_).argsort()[::-1][:10]
important_features  

In [ ]:
print(f"\nTop 10 Most Important Features (Lasso):")
for idx in important_features:
    print(f"- {features.columns[idx]}")

### Analyze GAM residuals

In [ ]:
gam_residuals = y_test - gam_pred
gam_residuals

### Explore residual distribution

In [ ]:
# Explore residual distribution (histograms, boxplots)
plt.hist(gam_residuals)
plt.title("GAM Residuals Distribution")
plt.show()

In [ ]:
print(f"\nThe most important features identified by Lasso include {', '.join(features.columns[important_features[:3]])}...")  # Briefly discuss top features


In [ ]:
# Example interpretation (replace with your specific insights)
if lasso_mse < gam_mse:
    print("\nLasso regression might be a better choice for this dataset based on MSE.")
else:
    print("\nGAM might be a better choice for this dataset based on MSE, potentially capturing non-linear relationships.")
